# Named entity recognition with Spacy


## Download libraries and models

In [73]:
import numpy as np
import pandas as pd
import spacy
import json
import tqdm
#https://www.newscatcherapi.com/blog/train-custom-named-entity-recognition-ner-model-with-spacy-v3
#https://spacy.io/usage/training#quickstart
#ner annotator: 'https://tecoholic.github.io/ner-annotator/'

## Load training data 

### Merger multiple .json files in resources

In [ ]:
# Doing, don't run this code
def merge_json_from_folder(file_paths):
    # Iterate over each file path
    for file_path in file_paths:
        # Check if file exists
        if os.path.exists(file_path):
            # Read JSON from file
            with open(file_path, 'r') as f:
                json_data = json.load(f)
    # Read contents of both JSON files
    with open(file1, 'r', encoding = 'UTF-8') as f1:
        json1 = json.load(f1)

    with open(file2, 'r', encoding = 'UTF-8') as f2:
        json2 = json.load(f2)

    text1 = json1.get("annotations", [])[0][0]
    text2 = json2.get("annotations", [])[0][0]
    entities1 = json1.get("annotations", [])[0][1].get('entities', [])
    entities2 = json2.get("annotations", [])[0][1].get('entities', [])
    dict1 = {'text': text1, 'entities': entities1}
    dict2 = {'text': text2, 'entities': entities2}
    Ans = {}
    Ans['classes'] = json1.get('classes',[])
    Ans['annotations'] = [dict1, dict2]
    # Ans['annotations'][0] = {'text': }
    # [json1.get("annotations", [])[0], json2.get("annotations",[])[0] ]
    #json1["annotations"] = json1.get("annotations", []) + json2.get("annotations",[])
    
    #merged_json will replace json1
    #json1["annotations"] = merged_json

    return Ans

In [65]:
def merge_json(file1, file2,):
    # Read contents of both JSON files
    with open(file1, 'r', encoding = 'UTF-8') as f1:
        json1 = json.load(f1)

    with open(file2, 'r', encoding = 'UTF-8') as f2:
        json2 = json.load(f2)

    text1 = json1.get("annotations", [])[0][0]
    text2 = json2.get("annotations", [])[0][0]
    entities1 = json1.get("annotations", [])[0][1].get('entities', [])
    entities2 = json2.get("annotations", [])[0][1].get('entities', [])
    dict1 = {'text': text1, 'entities': entities1}
    dict2 = {'text': text2, 'entities': entities2}
    Ans = {}
    Ans['classes'] = json1.get('classes',[])
    Ans['annotations'] = [dict1, dict2]
    # Ans['annotations'][0] = {'text': }
    # [json1.get("annotations", [])[0], json2.get("annotations",[])[0] ]
    #json1["annotations"] = json1.get("annotations", []) + json2.get("annotations",[])
    
    #merged_json will replace json1
    #json1["annotations"] = merged_json

    return Ans

### Load training_data

In [71]:
training_data = merge_json(r'Training_data\NER0.json', r'Training_data\NER1.json' )
training_data['annotations'][0]['text']

'Terminologies that was created by knowledge of members. PER: Andrew Ng, Steve Jobs, Ralph Merkle, Stuart Haber, W.Scott Stornetta, Satoshi Nakamoto, user, consumer, producer, developer, investor,  leader, Caroline Pham, commissioner, Gary Gensler, CEO. ORG: Oracle, Google, Meta, Apple, Amazon, Viettel, UNICEF, WTO, Multicoin Capital, Commodity Futures Trading Commission. LOC: New York, London, Paris, Tokyo, Shanghai, Peking, HongKong, New South Wales, US, UK, France, Japan, China. PRODUCT: Apps, website, applications. DATE: June 21st, September 15th, 01/01/2023, May 16th 2004, December 12th 2024, 2017, 2024, Oct. 1 . DATA_AI: Machine learning, Data, Artificial intelligence, Deep learning, Bigdata, database, cluster, regression, autoregressive, patterns. BLOCKCHAIN: Blockchain, Crypto currencies, Bitcoin, BTC, Dogecoin, smart contracts, block, hash function, peer-to-peer,P2P consensus mechanism, Cryptography, Distributed ledger, Proof-of-work, Proof-of-stake, Encryption, Digital signat

## Load model

In [75]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [76]:
from spacy.util import filter_spans

In [77]:
for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 22.40it/s]


In [6]:
text = training_data['annotations']['text']
labels = training_data['annotations']['entities']
doc = nlp.make_doc(text)
ents = []
for start, end, label in labels:
    span = doc.char_span(start, end, label=label, alignment_mode="contract")
    if span is None:
        print("Skipping entity")
    else:
        ents.append(span)
filtered_ents = filter_spans(ents)
doc.ents = filtered_ents
doc_bin.add(doc)
doc_bin.to_disk("training_data.spacy") # save the docbin object

Skipping entity


## Train model by terminal

In [78]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [79]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    261.57    0.00    0.00    0.00    0.00
100     200       1245.09  19990.27  100.00  100.00  100.00    1.00
200     400         53.61     34.73  100.00  100.00  100.00    1.00
300     600         81.85     29.48  100.00  100.00  100.00    1.00
400     800        117.76     42.20  100.00  100.00  100.00    1.00
500    1000        184.75     62.49  100.00  100.00  100.00    1.00
600    1200        227.47     51.30  100.00  100.00  100.00    1.00
700    1400        160.78     35.07  100.00  100.00  100.00    1.00
800    1600         66.14     12.17 

## Test new model 

In [80]:
nlp_ner = spacy.load("model-best")

In [102]:
import os
df = pd.read_csv(r"D:\Downloads\DS&AI\Team work\PVT\OOP-PROJECT-G4\src\main\resources\data\preprocessed_data.csv")
df.columns

Index(['Unnamed: 0', ' Article title', ' Content', ' Creation date', ' Author',
       ' Category', ' Tags'],
      dtype='object')

In [104]:
df = df[' Content']
df[0]

'federal reserve digital asset bank custodia access master account judge rule friday federal reserve bank discretion grant master account judge scott skavdahl district court district wyoming judgment news blow custodia bank sue federal reserve board governor federal reserve bank kansas city 2022 delay decision application central bank master account master account allow institution direct access feds payment system provide direct access uss money supply available financial institution master account force rely partner bank master account provide service custodias position correct effectively mean depository institution charter law regardless soundly craft entitle master account allow direct access federal financial system judge accord court document custodia allege federal reserve board violate administrative procedure act action arbitrary capricious abuse discretion otherwise accordance law custodia judge compel board issue master account apa govern federal agency develop issue rule c

In [110]:
from spacy import displacy
test = input()
doc = nlp_ner(test)
spacy.displacy.render(doc, style="ent", jupyter=True)

In [111]:
ans = {}
for ent in doc.ents:
    ans[ent.label_] = []
for ent in doc.ents:
    ans[ent.label_].append((ent.start_char, ent.end_char))
for label in ans: 
    print("\n" + label + ": ")
    for text in ans[label]: 
        print(text, end= "; ")


#save to json file 
file_name = "Predict_NER3.json" 
with open(file_name, 'w') as f: 
    json.dump(ans,f)


ORG: 
(0, 6); 
LOC: 
(8, 13); 

## Conclusion: 
### Results: 
Model failed successfully. The reasons for that, to my perspective: 
- The NER0.json file has no meanings. That make hardly any contributions to the training process
- Other data is too small
### Plans: 
- About 1st reason, with each tag, we have to find websites full of termilogies about that entity.
- Abour 2nd reasion, we should use some LLMs to create training_data for us
